# Python for Social Science

<img src="../figures/PySocs_banner.png" width="50%" align="left">

# Pandas Advanced Topics

Continuing from our previous class, we will explore a range of common data manipulation techniques in pandas, such as handling missing data, reshaping, pivoting, and merging.

## Handling Missing Data

![Missing Data](../figures/missing_data.png)

Missing data is a common issue in real-world data analysis. When dealing with data, it's important to correctly identify, manage, and understand missing values to ensure accurate analysis and interpretation.

In this section, we will cover:

- Understanding different types of missing data, `None`, `np.nan`, `pd.NA`
- Identifying missing data with `isna()`, `isnull()` and `notnull()`  
- Removing missing data using `dropna()`  
- Filling missing data with `fillna()` 
- Imputing missing data stochastically

### Missing Data Types

- `None`: Native Python null value, often used in **object**-type columns.
- `np.nan`: NumPy's "Not a Number", used for **float**-type columns.
- `pd.NA`: Pandas' newer missing value marker, introduced for better support across data types.

For illustration purposes, the dataset below includes all three types of missing values.

In [ ]:
import pandas as pd
import numpy as np

data = {
    'name': ['Alice', 'Bob', None, 'David'],
    'age': [25, np.nan, 30, 22],
    'income': [50000, 60000, pd.NA, 45000]
}
df = pd.DataFrame(data)
df

Pandas handles `None`, `np.nan`, and `pd.NA` with subtle but important differences. Here's a breakdown of how each behaves and how Pandas treats them:

### 🔍 Missing Value Types

| Value Type | Origin  | Typical Use       | Treated as Missing? | Notes                                |
|------------|---------|-------------------|----------------------|--------------------------------------|
| `None`     | Python  | Object dtype      | ✅ Yes               | Compatible with object columns       |
| `np.nan`   | NumPy   | Float dtype       | ✅ Yes               | Used in numerical columns            |
| `pd.NA`    | Pandas  | Nullable dtypes   | ✅ Yes               | Consistent across all data types     |


`None` is a sentinel value for native Python objects. It is compatible with NumPy arrays and Pandas Series as long as they are of `dtype=object`.

To clarify this point, let's create an NumPy array of integer values.

In [ ]:
num_arr = np.array([1, 2, 3, 4])
num_arr.dtype

If we attempt to assign `None` to one of the elements, a `TypeError` will occur because `num_arr` is of type 'int64', which cannot store values of type 'object' such as `None`.

❌: TypeError

```python
num_arr[2] = None
```

`np.nan` is NumPy's "Not a Number", which is used for **float**-type arrays. Because `num_arr` has already been defined as an integer array, the following will also raise a TypeError.

❌: TypeError

```python
num_arr[2] = np.nan
```

We **cannot** use `np.nan` to represent missing values in integer arrays because `np.nan` is a sentinel value for **float**-type. 

### Custom Markers of Missing Data

Unfortunately, NumPy arrays with the 'int64' dtype lack a built-in sentinel value to represent missing data.

Two common workarounds are: (1) defining the array with a 'float64' dtype, and (2) using a specific integer value, such as `-999`, as a sentinel. For example:

In [ ]:
# defining the array with a 'float64' dtype
num_arr = np.array([1, 2, 3, 4], dtype='float64')
num_arr[2] = np.nan
num_arr

In [ ]:
# using a specific integer value as a sentinel
num_arr = np.array([1, 2, 3, 4], dtype='int64')
num_arr[2] = -999
num_arr

If you use the second approach (e.g., `-999`), you should take precautions and carefully document them for others and for your future self to prevent doing something like this inadvertently:

In [ ]:
num_arr.mean()

One way to guard against such glitches is to create a **masked** array instead.

In [ ]:
arr = np.array([1, 2, -999, 4])
mask = [0, 0, 1, 0]

# alternatively
mask = [1 if x == -999 else 0 for x in arr]

masked_arr = np.ma.masked_array(arr, mask=mask, dtype='int64')
masked_arr

Explanation of `mask=[0, 0, 1, 0]`:
- `0` means not masked (valid data)
- `1` means masked (missing or invalid data)

So in this case:
- `1` → valid
- `2` → valid
- `-999` → masked (treated as missing)
- `4` → valid

In [ ]:
masked_arr.mean()

If you think that's too much work, the best option for now is to define your array as `float64` instead. 

In [ ]:
float_arr = np.array([1, 2, np.nan, 4], dtype='float64')
float_arr

In [ ]:
np.mean(float_arr) # or float_arr.mean()

❗ By default, `np.mean()` does not ignore `NaN` values — if a `NaN` is present anywhere in the data, the result will propagate `NaN`, making the entire mean calculation return `NaN`.

To compute the mean of a NumPy array that contains `np.nan` values, you need to use `np.nanmean()` instead of the regular `np.mean()`.

In [ ]:
np.nanmean(float_arr)

Alternatively, you can manually remove `NaN` values before calculating the mean:

In [ ]:
np.mean(float_arr[~np.isnan(float_arr)])

Unlike `np.array`, a Pandas Series can handle `np.nan` values by default.

In [ ]:
float_ser = pd.Series([1, 2, np.nan, 4], dtype='float64')

In [ ]:
float_ser.mean()

By default, `.mean()` ignores `NaN` values (`skipna=True`)

In [ ]:
float_ser.mean(skipna=False)

### Practice Exercise 1

You are given a list of daily temperatures in Celsius:

In [ ]:
temps_list = [22.5, 23.0, None, 21.5, 20.0, np.nan, 19.5]

Follow these steps:

1. Convert the list to a NumPy array with `dtype='float64'` to properly handle missing values (`None` and `np.nan`).
2. Identify any missing values in the array.
3. Calculate the average temperature (`mean_temps`), ignoring missing values.
4. Replace all missing values with the **mean** value to create a cleaned array (`temps_cleaned`), and display the result.

In [ ]:
# Step 1. Create the NumPy array `temps`
# YOUR CODE HERE


In [ ]:
# Step 2. Identify missing values `missing_mask` 
# YOUR CODE HERE


In [ ]:
# Step 3. Compute the average temperature, ignoring missing values, 
# as `mean_temps`
# YOUR CODE HERE


In [ ]:
# Step 4. Replace all missing values with the mean of the non-missing values, 
# and show the cleaned array, `temps_cleaned`.
# YOUR CODE HERE


### Pandas Nullable Dtype

More recently, Pandas introduced a newer missing value marker, `pd.NA`, for better support across different data types. For example, you can create a **pd.Series** with all available markers of missing data.

In [ ]:
num_arr = pd.Series([1, None, np.nan, pd.NA], dtype="Int64")
num_arr

These *nullable dtypes* are distinguisehd from regular dtypes by capitalization of their names (e.g., `pd.Int64` vs. `np.int64`).

### Identifying Missing Data

Python doesn't have built-in functions for identifying missing values. However, Pandas provides two functions for checking all missing value types (`None`, `np.nan`, `pd.NA`), namely `pd.isnull()` and `pd.isna()`, and also corresponding methods, e.g., `DataFrame.isnull()` and `DataFrame.isna()`. 

`pd.isna()` is synomyous to and alias for `pd.isnull()`, and they are functionally equivalent.

In [ ]:
pd.isnull(df)

In [ ]:
df.isna()

In [ ]:
df.notna()

In [ ]:
~df.isna() # equivalent to df.notna()

#### Filtering Out Missing Data

Dropping missing values is straightforward—sometimes deceptively so, as it may lead to careless application. To illustrate this, let's create a sample DataFrame.

In [ ]:
# Set a random seed for reproducibility
np.random.seed(2222)

# Create a sample data (dictionary) with missing values
data = {
    'CustomerID': [f'CUST{i:03d}' for i in range(1, 11)],
    'Age': [25, 34, np.nan, 45, 29, 38, np.nan, 50, 41, 34],
    'AnnualIncome': [50000, 62000, 58000, np.nan, 72000, 69000, 64000, 71000, np.nan, 67000],
    'SatisfactionScore': [8, 7, 9, 6, np.nan, 8, 7, np.nan, 6, 9],
    'LastPurchaseDate': [
        '2023-01-15', '2023-02-20', '2023-03-05', np.nan,
        '2023-04-10', '2023-05-25', '2023-06-30', '2023-07-15',
        np.nan, '2023-08-20'
    ]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Convert LastPurchaseDate to datetime
df['LastPurchaseDate'] = pd.to_datetime(df['LastPurchaseDate'], errors='coerce')
# errors='coerce' prevents the code from crashing due to unexpected formats

df

In [ ]:
# Number of missing values by column
df.isna().sum() # equivalent to df.isna().sum(axis=0)

In [ ]:
# Number of missing values by row
df.isna().sum(axis=1)

In [ ]:
# ⚠️ List-wise deletion
df.dropna()

By default, `dropna()` will remove all rows that contain any missing values (i.e., it performs list-wise deletion). This is equivalent to:

In [ ]:
df.dropna(how="any")

You may want to drop rows with all NA values:

```python
df.dropna(how="all")
```

Alternatively, you may want to set a threshold for **a minimum number** of non-NA values for the row to be kept:

In [ ]:
df.dropna(thresh=4)

`dropna()` also supports the `axis=1` argument, which allows you to apply the previously specified rules to columns. For example,

In [ ]:
df.dropna(how="any", axis=1)

⚠️ This removed all columns containing any missing data, leaving almost no information except for the ID field.

### Filling In Missing Data

Instead of dropping rows or columns with missing data, we might want to replace them with specific values. Pandas offers the `fillna` method, which can **easily** fill in the gaps. Be cautious, because you might accidentally introduce incorrect data and undermine your otherwise good results!

In [ ]:
df['Age'].fillna(999)

## 🧠 Imputing Missing Data

Imputation is the process of replacing missing data with substitute values. It is an important step in data cleaning and preprocessing, especially since real-world datasets often have missing values caused by errors, non-responses, censoring, or system limitations.

Without imputation, many analytical methods—such as regression, classification, or clustering—may fail or yield biased results.

#### 🔧 Common Imputation Techniques

- **Mean Imputation**
    Replace missing values with the column's mean. Simple and fast, but can distort (decrease) variance.
- **Median Imputation**
    Use the median instead of the mean. More robust to outliers.
- **Mode Imputation**
    Replace with the most frequent value. Useful for categorical data.
- **Random Imputation (with probabilities)**
    Sample from existing values based on their observed frequencies. Helps preserve the original distribution.
- **Model-based Imputation**
    Use predictive models (e.g., regression, k-NN, or machine learning) to estimate missing values.

In [ ]:
# Mean imputation

df['Age_inp'] = df['Age'].fillna(df['Age'].mean())
df[['Age', 'Age_inp']]

In [ ]:
# Median imputation

df['Age'].fillna(df['Age'].median())

In [ ]:
df['Age'].mode()

In [ ]:
# Mode imputation

df['Age'].fillna(df['Age'].mode()[0])

Subsetting the output of `mode()` using `mode()[0]` is necessary because the function can return an array instead of a scalar value when there are ties.

In [ ]:
# Random imputation
probs = df['SatisfactionScore'].value_counts(normalize=True)

In [ ]:
probs.index

In [ ]:
probs.values

This function `random_impute()` implements a random imputation based on the observed probabilities.

In [ ]:
def random_impute(arr):
    probs = arr.value_counts(normalize=True)
    return arr.apply(lambda x: x if pd.notnull(x) else np.random.choice(probs.index, p=probs.values))


In [ ]:
df['SatisfactionScore_inp'] = random_impute(df['SatisfactionScore'])
df

Replacing missing values in this manner is acceptable in certain situations, particularly when the proportion of missing data is relatively small (e.g., less than 5%). However, more advanced methods exist that account for and properly represent the uncertainty of imputed values and their conditional dependency on other variables.

### Practice Exercise 2

For this exercise, we will download the census income dataset (`adult.data`) from the UCI Machine Learning Repository. We'll select a subset of the data and artificially add missing values to some continuous or ordinal variables for imputation practice.

In [3]:
# Download `adult.data` from UCI ML repo or via URL
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
cols = [
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
    "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss",
    "hours_per_week", "native_country", "income_gt_50k"
]

df = pd.read_csv(url, header=None, names=cols, na_values=" ?", skipinitialspace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income_gt_50k   32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [ ]:
# Filtering
df_sample = df.loc[df["native_country"] == 'United-States']

In [ ]:
# Subsetting
rng = np.random.default_rng(seed=2222)

df_sample = df.sample(n=1000)
df_sample_imp = df_sample.copy()

In [ ]:
# Introducing missing data
# 8% missing in age, 5% in hours_per_week, 10% in capital_gain, 10% in capital_loss
for col, pct in [("age", 0.08), ("hours_per_week", 0.05), ("capital_gain", 0.10), ("capital_loss", 0.10)]:
    mask = rng.random(len(df_sample_imp)) < pct
    df_sample_imp.loc[mask, col] = np.nan

In [ ]:
# Counting NA
df_sample_imp.isna().sum(axis=0)

**Task A**. Calculate and print the mean and median for the variables `age`, `hours_per_week`, `capital_gain`, and `capital_loss`, excluding any missing values. Begin by listing the columns you want to analyze, and store the resulting statistics in arrays.

In [ ]:
# YOUR CODE HERE


**Task B**. Impute missing values in:

- `age` and `hours_per_week` using **mean** imputation
- `capital_gain` and `capital_loss` using **median** imputation

In [ ]:
# YOUR CODE HERE


**Task C**. After imputation, re-check that there are no missing values and compute descriptive statistics (mean, median, std) for all columns.

In [ ]:
# YOUR CODE HERE


**Task D**. Compare how the distribution of the imputed columns, `cols`, changes before vs. after imputation. Hint: `describe()`

In [ ]:
# YOUR CODE HERE


## Multiple Imputation

Multiple Imputation (MI) is a statistical technique for handling missing data by creating several plausible datasets, analyzing each one, and then combining the results.

| **Single Imputation**                                | **Multiple Imputation**                                                    |
| ---------------------------------------------------- | -------------------------------------------------------------------------- |
| Fill missing values once (e.g., mean, median, mode). | Create *M* datasets with different plausible values for each missing cell. |
| Ignores uncertainty → **underestimates variance**.   | Captures uncertainty → **more realistic statistical inference**.           |
| Easy but **biased estimates** possible.              | More robust, theoretically sound.                                          |

### Why Imputation is Needed

Real-world datasets often contain missing values due to:
- Non-responses in surveys
- Data entry errors
- Merge mismatches

Many statistical models (e.g., linear regression, logistic regression, tree-based models) cannot handle missing values directly, so we must fill (impute) them.

### Workflow of Multiple Imputation

- **Imputation step**: 
Generate several complete datasets by filling missing values with random draws from predictive distributions.

- **Analysis step**: 
Perform the desired analysis on each completed dataset separately.

- **Pooling step**: 
Combine results using Rubin's Rules to get final parameter estimates and standard errors.

![Source: ](../figures/MI.png)

### Python Libraries for Multiple Imputation

| Library        | Method Type                                     | Notes                          |
| -------------- | ----------------------------------------------- | ------------------------------ |
| `scikit-learn` | Iterative Imputer (MICE-like)                   | Good for predictive modeling   |
| `statsmodels`  | MICE (Multiple Imputation by Chained Equations) | Classical statistical approach |
| `fancyimpute`  | Advanced ML-based imputation                    | Deep learning, KNN             |
| `miceforest`   | Random forest-based MI                          | Very flexible, robust          |


### Example Dataset: Customer Survey

Let's simulate a dataset (N=100) with missing values. 

In [ ]:
# Set random seed for reproducibility
np.random.seed(2222)

# Create dataset
data = pd.DataFrame({
    'Age': np.random.randint(18, 65, 100),
    'Income': np.random.normal(55000, 15000, 100).round(0),
    'SpendingScore': np.random.randint(1, 100, 100),
    'Satisfaction': np.random.choice(['Low', 'Medium', 'High'], 100)
})

data.head()

In [ ]:
data.describe()

In [ ]:
data['Satisfaction'].value_counts()

We will randomly select 10% of the `Age` data and replace them with `np.nan`. Similarly, 15% of `Income` and 5% of `SpendingScore` data will be randomly replaced with missing values.

In [ ]:
# Introduce missingness
data.loc[data.sample(frac=0.10).index, 'Age'] = np.nan
data.loc[data.sample(frac=0.15).index, 'Income'] = np.nan
data.loc[data.sample(frac=0.05).index, 'SpendingScore'] = np.nan

print("Missing value counts:\n")
print(data.isnull().sum(), "\n")

data.head()

In [ ]:
data.describe()

In [ ]:
data['Satisfaction'].value_counts()

#### Visualizing Missing Values

We can visualize the missing data patterns using a heatmap. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def missing_pattern(df):
    plt.figure(figsize=(6, 4))
    sns.heatmap(df.isnull(), cbar=False, cmap="Oranges", yticklabels=False)
    plt.title("Missing Data Heatmap")
    plt.show()
    
missing_pattern(data)

#### MICE with `miceforest`

Multiple Imputation by Chained Equations (MICE) addresses missing data in a dataset by repeatedly running predictive models that estimate and fill in the missing values. During each cycle, each chosen variable is imputed based on the information from all other variables. This process is repeated until the results suggest convergence has been achieved.

![miceforest](../figures/miceforest.png)

Source: https://pypi.org/project/miceforest/

#### Data Pre-Processing

One retriction in running `miceforest` is that you cannot directly impute object columns.

It will raise an error if your input DataFrame contains columns with object dtype (e.g., strings or mixed types), as `miceforest` only supports Numeric types (int, float) and Categorical types (category in pandas).

In [ ]:
data.dtypes

We can convert `Satisfaction` to a categorical variable as follows:

In [ ]:
data['Satisfaction'] = data['Satisfaction'].astype('category')
data.dtypes

In [ ]:
data.isna().sum(axis=0)

When imputing a variable with missing data (say, "Age"), MICEforest:
- Treats "Age" as the target variable.
- Uses all the other variables in the dataset as predictors ("Income" and "SpendingScore").
- Trains a random forest model to predict the missing values of "Age" based on the observed data.

For example:

If "Age" has missing values:
- MICEforest builds a random forest model:

$$Age ∼ Education + Income + SpendingScore$$

- It uses rows where "Age" is NOT missing to train the model.
- Then, it predicts "Age" for rows where it is missing.

This process is repeated for every column with missing data, one variable at a time.

**Step 0** – Initialization

- All missing values are temporarily filled in using a simple method:
    - Mean imputation, median imputation, or random sampling.
- This gives you a complete dataset to start modeling, even though the initial imputations are crude.

**Step 1** – First Iteration

MICE now goes column-by-column, imputing missing values:

1. Impute Age
    - Model: Age ~ Income + SpendingScore
    - Uses current versions of Income and Education (including their temporary imputed values).

2. Impute Income

- Model: Income ~ Age + SpendingScore
- Uses updated Age and current SpendingScore.

3. Impute SpendingScore

- Model: SpendingScore ~ Age + Income
- Uses updated versions of Age and Income.

At the end of **Step 1**, all variables have new, better imputations.

**Step 2** – Repeat

- The process repeats again, now using the updated imputations as predictors.
- With each iteration, the imputations become more consistent, as each variable's prediction benefits from improved values of the others.

In [ ]:
import miceforest as mf

# Create a kernel and run multiple imputations
kernel = mf.ImputationKernel(
    data,
    num_datasets=4,
    save_all_iterations_data=True,
    random_state=2222
)

# Run the MICE algorithm for 5 iterations
kernel.mice(5)

# Printing the kernel will show you some high level information.
print(kernel)

After we have run mice, we can obtain our completed dataset directly from the kernel:

In [ ]:
kernel.datasets

In [ ]:
data_imp_0 = kernel.complete_data(dataset=0) # 0 = first imputed dataset

missing_pattern(data_imp_0)

In [ ]:
data_imp_0.isnull().sum()

Performing statistical analyses on multiple imputed datasets requires concatenating (stacking) all the imputed sets into a single DataFrame. We will cover this topic in more detail later in the semester. For now, we will demonstrate how to prepare a concatenated DataFrame in long format. 

In [ ]:
# Stack all imputed datasets into a single DataFrame
data_imp_all = pd.concat(
    [kernel.complete_data(dataset=i).assign(_imputation_=i) 
     for i in kernel.datasets],
    ignore_index=True
)

In [ ]:
data_imp_all.shape

In [ ]:
data_imp_all.head()

In [ ]:
data_imp_all.tail()

Note that we use `pd.concat()` with a list comprehension because there isn't a built-in function in `miceforest` that combines all imputed datasets into a single DataFrame, similar to R’s `mice::complete(m="long")`.

To streamline this process for repeated use, we can create a custom function and add it to our toolbox:

In [ ]:
def combine_imp(kernel):
    """Combine all imputed datasets from miceforest kernel into one DataFrame."""
    return pd.concat(
        [kernel.complete_data(dataset=i).assign(_imputation_=i) 
         for i in kernel.datasets],
        ignore_index=True
    )

# Usage
data_imp_all = combine_imp(kernel)

In [ ]:
data_imp_all["_imputation_"].value_counts()

In [ ]:
data_imp_all.groupby("_imputation_").describe().round(1).T

### Practice Exercise 3

Perform multiple imputation on the data using `miceforest` and the following steps:
- Generate three (3) imputed datasets.
- Run the algorithm with three (3) iterations.
- Set the random seed to 2222.
- Stack the imputed datasets vertically to create `df_inp_combined`.
- For each imputation, produce descriptive statistics for the variable of interest.

In [ ]:
# Example dataset
df = pd.DataFrame({
    'Age': [25, np.nan, 30, 35, np.nan, 22, 26, 31, np.nan, 29],
    'BMI': [22.5, 24.0, np.nan, 27.0, 26.5, 21.9, 25.2, np.nan, 23.0, 24.5]
})

df

In [ ]:
# YOUR CODE HERE


## Hierarchical Indexing

Hierarchical indexing (also called MultiIndex) allows you to have multiple levels of indices in your DataFrame or Series. This is useful for working with higher-dimensional data in a 2D structure. 


📊 State Population Estimates

| State          | July 1, 2014 Population | July 1, 2024 Population |
| -------------- | ----------------------- | ----------------------- |
| **California** | 38,680,810              | 39,431,263              |
| **New York**   | 19,378,124              | 19,867,248              |
| **Texas**      | 26,094,422              | 31,290,831              |


In [ ]:
# Define index
index = [
    ('California', 2014),
    ('California', 2024),
    ('New York', 2014),
    ('New York', 2024),
    ('Texas', 2014),
    ('Texas', 2024)
]

# State population estimates
population_values = [38_680_810, 
                     39_431_263, 
                     19_378_124, 
                     19_867_248, 
                     26_094_422, 
                     31_290_831]

You may consider using the tuples as defined in `index` as keys.

In [ ]:
state_population = pd.Series(population_values, index=index)
state_population

With this indexing scheme, you can select a specific (state, year) pair or slice the data accordingly.

In [ ]:
state_population[("Texas", 2014)]

In [ ]:
state_population[("Texas", 2014):("Texas", 2024)]

However, selecting all population estimates for 2023 (or all population estimates for Texas) is no longer straightforward. You can still retrieve the data, but the process is now more complicated:

In [ ]:
state_population[[i for i in state_population.index if i[1] == 2014]]

In [ ]:
state_population[[i for i in state_population.index if i[0] == "Texas"]]

Pandas offers a simpler method for indexing hierarchically structured data points: `pd.MultiIndex`

In [ ]:
multi_index = pd.MultiIndex.from_tuples(index, names=['State', 'Year'])
multi_index

In [ ]:
# Create the Series
state_population = pd.Series(population_values, index=multi_index)
state_population

Now, we can access all data for Texas as easily as this:

In [ ]:
state_population["Texas", :]

Or, all data for 2024:

In [ ]:
state_population[:, 2024]

In [ ]:
state_population_state = state_population.unstack()
state_population_state

We can also choose which level to "unstack" using the level parameter in `unstack()`. It can be either the level name or the level number.

In [ ]:
state_population_year= state_population.unstack(level="State") # (level=0)
state_population_year

You can easily undo "unstack" with `stack()`.

In [ ]:
state_population_state.stack()

We can create a new variable `Gain` and then stack:

In [ ]:
state_population_state['Gain'] = state_population_state[2024] - state_population_state[2014]
state_population_state.stack()

For more details, see Chapter 17 of VanderPlas (2023), pages 132-144.

## Pivot Tables

Pivot tables are one of the most powerful tools in Pandas for summarizing and aggregating data. They allow you to aggregate, group, and reshape your data in a single step.

#### 1. What is a Pivot Table?

A pivot table:
- Summarizes data by grouping rows and aggregating values.
- Similar to pivot tables in Excel but more flexible.
- Great for multi-dimensional summaries like:
    - Average age by gender and passenger class
    - Total sales by region and quarter
    - Count of events per category and time period

**Basic syntax**:

```python
pd.pivot_table(
    data, 
    values=None,      # column(s) to aggregate
    index=None,       # row grouping
    columns=None,     # column grouping
    aggfunc='mean'    # aggregation function: mean, sum, count, etc.
)
```

#### 2. Loading the Titanic Dataset

We will use the Titanic dataset, which contains information about passengers on the Titanic:

- `survived`: Whether the passenger survived (1) or not (0)
- `pclass`: Passenger class (1 = First, 2 = Second, 3 = Third)
- `sex`: Gender (female, male)
- `age`, `fare`: Numerical values
- `embarked`: Port of embarkation (C, Q, S)
- etc.

In [ ]:
titanic = sns.load_dataset('titanic')
titanic.info()

#### 3. Simple Pivot Table: Survival Rate by Gender

Goal: Find the survival rate of male and female passengers.

In [ ]:
titanic.pivot_table(index=['sex'], values=['survived'], aggfunc='mean').round(2)

#### 4. Adding a Second Grouping Level

Goal: Survival rate by gender and passenger class.

In [ ]:
titanic.pivot_table(
    values='survived',
    index='sex',
    columns='pclass',
    aggfunc='mean'
).round(2)

#### 5. Adding Marginal Means

Goal: Survival rate by gender and passenger class, adding the `All` values without taking into account gender/passenger class.

In [ ]:
titanic.pivot_table(
    values='survived',
    index='sex',
    columns='pclass',
    aggfunc='mean',
    margins=True
).round(2)

#### 6. Multiple Values

Goal: Survival rate and average fare by gender and passenger class, adding the `All` values without taking into account gender/passenger class.

In [ ]:
titanic.pivot_table(
    values=['survived', 'fare'],
    index='sex',
    columns='pclass',
    aggfunc='mean',
    margins=True
).round(2)

#### Adding Age after Binning

We can include the `age` variable as another index. Since it is currently a continuous variable, we can divide it into discrete intervals, or "bins," for analysis: (0, 20], (20, 30], (30, 40], (40, 50], and (50, 100].

In [ ]:
bins = [0, 20, 30, 40, 50, 100]
titanic['age_category'] = pd.cut(titanic.age, bins)
titanic['age_category'].value_counts(sort=False) #By default, Series.value_counts() sorts by counts

🔧 Now, we can create a nested index using both `sex` and `age_category` as follows:

In [ ]:
titanic.pivot_table(
    values=['survived'],
    index=['sex', 'age_category'],
    columns='pclass',
    aggfunc=['count', 'mean'],
    observed=False
).round(2)

![Combining DataFrames](../figures/join.png)

## Combining and Joining DataFrames

In real-world data analysis, information is often spread across multiple datasets. For example:
- A student dataset might store student demographic and test score details,
- A teacher dataset might contain teacher information, and
- A school dataset might contain school details.

To analyze these datasets together, Pandas provides powerful tools to combine, merge, and join them.

#### 1. Why Combine DataFrames?

Typical use cases:
- **Enriching data**: Adding student demographics to test score records.
- **Cleaning data**: Combining multiple district or school files or reports into a single table.
- **Analyzing relationships**: Comparing variables from different sources.

Pandas supports several methods for combining DataFrames:

| Method                      | Use Case                                     |
| --------------------------- | -------------------------------------------- |
| `pd.concat()`               | Stacking datasets vertically or horizontally |
| `df.merge()` / `pd.merge()` | SQL-style joins (inner, outer, left, right)  |
| `df.join()`                 | Simplified joins using indexes               |


#### Example Datasets

For illustration, we will work with these sample datasets:

In [ ]:
# Student datasets
students = pd.DataFrame({
    'student_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Emily'],
    'grade': [3, 3, 4, 4, 4]
})

demog = pd.DataFrame({
    'student_id': [4, 3, 2, 1],
    'age': [10, 9, 8, 8],
    'sex': ['F', 'M', 'M', 'F']
})

students, demog

In [ ]:
# Test datasets
scores_fall = pd.DataFrame({
    'test_id': ["Fall"] * 4,
    'student_id': [2, 1, 3, 4],
    'score': [250, 450, 300, 200]
})

scores_winter = pd.DataFrame({
    'test_id': ["Winter"] * 5,
    'student_id': [2, 1, 3, 4, 5],
    'score': [270, 460, 320, 210, 510]
})

scores_spring = pd.DataFrame({
    'test_id': ["Spring"] * 5,
    'student_id': [1, 3, 4, 5, 6],
    'score': [470, 360, 220, 500, 410]
})

scores_fall, scores_winter, scores_spring


### Concatenation (`pd.concat`)

pd.concat() is used to stack DataFrames:
- Vertical stack (rows) → similar to adding records from another file.
- Horizontal stack (columns) → similar to adding new variables or attributes.

In [ ]:
# Vertical Concatenation
# Combining `scores_fall` and `scores_spring`

scores = pd.concat([scores_fall, scores_winter, scores_spring], ignore_index=True)
scores

**Note**: `ignore_index=True` resets the index after concatenation.

In [ ]:
scores.groupby('test_id')['score'].mean()

By default, `groupby()` in pandas sorts the group keys alphabetically (or numerically).

You can keep the original order of appearance by setting the parameter:

In [ ]:
scores.groupby('test_id', sort=False)['score'].mean()

In [ ]:
# Horizontal concatenation
# Adding new columns from a separate datasets
# ⚠️ This is problematic!

pd.concat([students, demog], axis=1)

⚠️`pd.concat(axis=1)` aligns records based solely on their **index**, not by columns as `pd.merge()` does. 

While you can use `reindex` to manually align records, it is generally better to use `merge()` or `join()` when you need to align data on key columns.

In [ ]:
pd.concat([students, demog.reindex([3, 2, 1, 0]).reset_index(drop=True)], axis=1)

### Merging DataFrames (`merge`)

The `merge()` function works like SQL joins by combining datasets based on one or more keys.

Let's repeat the previous horizontal concatenation example, but this time we'll use `merge()` with `student_id` as the key.

In [ ]:
# Inner Join
pd.merge(students, demog, on='student_id', how='inner')

Only students who have demographic information are retained.

In [ ]:
# Outer Join
pd.merge(students, demog, on='student_id', how='outer')

Outer join keeps all records from both datasets.

In [ ]:
# Jeft Join
pd.merge(students, demog, on='student_id', how='left')

A left join retains all records from the left table, displaying missing demographic information as `NaN`.

In [ ]:
# Right Join
pd.merge(students, demog, on='student_id', how='right')

A right join keeps all records from the right table.

### Pivoting "Long" to "Wide"

We often store multiple time-varying datasets in a *long* or *stacked* format. For example, in the `scores` dataset, each row represents an individual’s test score at a specific occasion, rather than storing multiple scores per student in separate columns.

In [ ]:
scores

In [ ]:
scores_wide = scores.pivot(index="student_id", 
                           columns="test_id",
                           values="score")
scores_wide

By omitting the last argument, we can create hierarchical columns.

In [ ]:
scores.pivot(index="student_id", 
             columns="test_id")

### Pivoting "Wide" to "Long"

We can reverse the "Long" to "Wide" transformation done by `.pivot()` using the `.melt()` function. `.melt()` collapses multiple columns into a single one, resulting in a DataFrame that is $k$ times longer than the input, where $k$ is the number of columns being collapsed.

In [ ]:
scores_wide

`student_id` is currently set as the index. Before collapsing the columns, we can convert `student_id` back to a column in the DataFrame by using `reset_index()` as shown below.

Note that by default, `reset_index()` does NOT modify the DataFrame in place. It returns a new DataFrame with the reset index. If you want to modify the original DataFrame you must assign the result back:

```python
scores_wide = scores_wide.reset_index()
```

or use 

```python
scores_wide.reset_index(inplace=True)
```

In [ ]:
scores_wide.reset_index(inplace=True)

Now, we can use `melt` with "student_id" as the identifier variable and gather all test score variable names into a "Test" column, with their corresponding values placed in a "Score" column.

In [ ]:
scores_wide.melt(id_vars="student_id",
                 value_vars=["Fall", "Winter", "Spring"],
                 var_name="test_id",
                 value_name="score")

### Wrap Up

That's all for now.
- Please complete the DC course "Introduction to Data Visualization with Matplotlib" by noon on 10/6.
- Submit the in-class exercise notebook by 6:00 PM today.

BY PRINTING YOUR NAME BELOW, YOU CONFIRM THAT THE EXERCISES YOU SUBMITTED IN THIS NOTEBOOK ARE YOUR OWN AND THAT YOU DID NOT USE AI TO ASSIST WITH YOUR WORK.

In [ ]:
# PRINT YOUR NAME
print("Enter Your Name Here")